In [30]:
import numpy as np
import pywt
w_name = 'bior5.5'
simple_signal = np.array([1, 2, 3, 5, 12, 9, 8, 2, 4, 5])
coeffs_pywt = pywt.wavedec(simple_signal, w_name, level=1)
wavelet = pywt.Wavelet(w_name)
low_pass_reconstruction = wavelet.rec_lo
high_pass_reconstruction = wavelet.rec_hi
pywt.waverec(coeffs_pywt,w_name,'smooth')

/home/fanghz/.local/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 1 is too high: all coefficients will experience boundary effects.
  warnings.warn(


array([ 1.,  2.,  3.,  5., 12.,  9.,  8.,  2.,  4.,  5.])

In [63]:
import numpy as np
def custom_convolve(signal, filter):
    signal_len = len(signal)
    filter_len = len(filter)
    output_len = signal_len + filter_len - 1
    output = np.zeros(output_len)
    for i in range(signal_len):
        for j in range(filter_len):
            output[i + j] += signal[i] * filter[j]
    return output

def custom_valid_convolve(a, v):
    Na = len(a)
    Nv = len(v)
    N = Na - Nv + 1  # 'valid' 模式下输出长度
    y = np.empty(N)
    v_flipped = v[::-1]  # 翻转卷积核

    for i in range(N):
        y[i] = np.sum(a[i:i+Nv] * v_flipped)
    return y

# 上采样：在元素之间插入零
def upsample(signal, factor=2):
    upsampled = np.zeros(factor * len(signal))
    upsampled[1::factor] = signal  # 将原始信号放在每隔factor的奇数位置
    return upsampled

# IDWT实现
def idwt(approx_coeffs, detail_coeffs, wavelet_filters):
    # 从低频重构滤波器和高频重构滤波器中获取重构滤波器
    low_pass_reconstruction, high_pass_reconstruction = wavelet_filters
    
    current_signal = approx_coeffs  # 初始化为近似系数

    # 从最高层开始逐层重构
    for detail in detail_coeffs:
        # 对近似系数和细节系数进行上采样
        upsampled_approx = upsample(current_signal)
        upsampled_detail = upsample(detail)

        # 分别与低通滤波器和高通滤波器进行卷积
        recon_approx = custom_valid_convolve(upsampled_approx, low_pass_reconstruction)
        recon_detail = custom_valid_convolve(upsampled_detail, high_pass_reconstruction)
        # recon_approx = np.convolve(upsampled_approx, low_pass_reconstruction,"valid")
        # recon_detail = np.convolve(upsampled_detail, high_pass_reconstruction,"valid")

        # 将近似和细节部分相加，得到新的重构信号
        current_signal = recon_approx + recon_detail

    return current_signal

# 示例调用
# 这里的 coeffs_pywt 和 wavelet_filters 需要根据你的具体情况定义
reconstructed_signal_manual = idwt(coeffs_pywt[0], coeffs_pywt[1:], (low_pass_reconstruction, high_pass_reconstruction))
print(reconstructed_signal_manual)


[ 1.  2.  3.  5. 12.  9.  8.  2.  4.]


In [31]:
upsampled_approx = upsample(coeffs_pywt[0])
upsampled_detail = upsample(coeffs_pywt[1])

In [32]:
upsampled_approx

array([ 0.  , 13.31,  0.  ,  4.4 ,  0.  ,  2.24,  0.  ,  3.12,  0.  ,
        8.46,  0.  , 13.49,  0.  ,  5.35,  0.  ,  7.22,  0.  ,  5.32,
        0.  ,  9.22])

In [37]:
sum(upsampled_approx[0:12] * low_pass_reconstruction)

1.2531131378327514

In [49]:
sum(np.pad(upsampled_approx,(2,0),"symmetric")[0:12]*low_pass_reconstruction)

2.6724185065996737

In [46]:
custom_convolve(upsampled_approx, low_pass_reconstruction) + custom_convolve(upsampled_detail, high_pass_reconstruction)

array([ 0.  ,  0.18, -0.04, -1.86, -1.24,  5.95, 12.42,  6.24,  3.02,
        1.93,  1.  ,  1.  ,  2.  ,  3.  ,  5.  , 12.  ,  9.  ,  8.  ,
        2.  ,  4.  ,  5.  ,  4.82,  4.04,  3.86,  9.24,  3.05, -0.42,
       -1.24, -0.02,  0.07,  0.  ])

In [47]:
custom_convolve(upsampled_approx, low_pass_reconstruction)

array([ 0.  ,  0.18, -0.04, -1.76, -1.26,  5.77, 11.55,  8.18,  2.49,
        1.03,  1.25,  1.16,  1.75,  3.5 ,  6.04,  9.44, 10.82,  6.96,
        2.84,  3.66,  5.44,  4.17,  3.23,  6.02,  7.78,  3.77, -0.88,
       -1.19, -0.02,  0.12,  0.  ])

In [16]:
reconstructed_signal_manual = idwt(coeffs_pywt[0], coeffs_pywt[1:], (low_pass_reconstruction, high_pass_reconstruction))
reconstructed_signal_manual[:len(reconstructed_signal_manual) - len(low_pass_reconstruction) + 2]


array([ 0.  ,  0.01, -0.  ,  0.09, -0.14,  0.13, -1.06,  4.66, -1.72,
        4.36, -0.35,  4.6 ,  1.06])

In [20]:
coeffs_pywt = [[1,2,3,4,5,6],[5,4,3,2,1,6]]
pywt.idwt(coeffs_pywt[0],coeffs_pywt[1],w_name,"smooth")

array([4.6 , 1.06])